In [ ]:
from utils import *
from sklearn import metrics
from sklearn.datasets import make_friedman2, make_friedman3
import pandas as pd

In [ ]:
datasets = []
datasets_std = []

In [ ]:
name = 'Friedman 2'
noise = 5

X_train, y_train = make_friedman2(noise=0.0, random_state=18397425)
_, y_train_noisy = make_friedman2(noise=noise, random_state=18397425)
X_norm_train= normalize(X_train, axis=1)

X_test, y_test = make_friedman2(noise=0.0, random_state=30189745)
_, y_test_noisy = make_friedman2(noise=noise, random_state=30189745)
X_norm_test= normalize(X_test, axis=1)

# [X_train, y_train, X_test, y_test, X_draw, norm, noise, name]
datasets.append([X_norm_train, y_train, X_norm_test, y_test, X_test, True, 0.0, name])
datasets.append([X_norm_train, y_train_noisy, X_norm_test, y_test_noisy, X_test, True, 0.15, name])

for i in range(0, X_train.shape[1]):
    X_train[:,i] = (X_train[:,i] - np.mean(X_train[:,i])) / np.std(X_train[:,i])
    X_test[:,i] = (X_test[:,i] - np.mean(X_train[:,i])) / np.std(X_train[:,i])

X_norm_train = normalize(X_train, axis=1)
X_norm_test = normalize(X_test, axis=1)

datasets_std.append([X_norm_train, y_train, X_norm_test, y_test, X_test, True, 0.0, name + ' Std'])
datasets_std.append([X_norm_train, y_train_noisy, X_norm_test, y_test_noisy, X_test, True, 0.15, name + ' Std'])

In [ ]:
depths = (3, 25, 100)
experiment_output = []
for dataset in datasets:
    for depth in depths:
        experiment_output.append(experiment(dataset, depth=depth))

In [ ]:
depths = (3, 25, 100)
experiment_output_std = []
for dataset in datasets_std:
    for depth in depths:
        experiment_output_std.append(experiment(dataset, depth=depth))

In [ ]:
depths=(3,25,100)
arrays = [
    ['Friedman 2', 'Friedman 2 Std'],
    [False, True],  # Noise
]

index = pd.MultiIndex.from_product(
    arrays, names=["Dataset", "Noise"])


df_gaus_sd = pd.DataFrame(
    index=index,
    columns=['D3rmse', 'D25rmse',  'D100rmse', 'D3corr', 'D25corr', 'D100corr'])

df_lap_sd = pd.DataFrame(
    index=index,
    columns=['D3rmse', 'D25rmse',  'D100rmse', 'D3corr', 'D25corr', 'D100corr'])

df_gaus_sd

In [ ]:
for exp in experiment_output:
    depth = exp['kernel']['depth']
    name = exp['dataset']['name']
    noise = exp['dataset']['noise']

    if noise==0.0: 
        noisy = False 
    else: 
        noisy = True

    if name == 'Nonpolynomial':
        name = 'Nonpoly'

    if exp['dataset']['norm']:
        df_lap_sd[f'D{depth}rmse'][name, noisy]  = exp['lap']['pred_rmse']
        df_lap_sd[f'D{depth}corr'][name, noisy]  = exp['lap']['pred_corr']
        df_gaus_sd[f'D{depth}rmse'][name, noisy] = exp['gaus']['pred_rmse']
        df_gaus_sd[f'D{depth}corr'][name, noisy] = exp['gaus']['pred_corr']

for exp in experiment_output_std:
    depth = exp['kernel']['depth']
    name = exp['dataset']['name']
    noise = exp['dataset']['noise']

    if noise==0.0: 
        noisy = False 
    else: 
        noisy = True

    if name == 'Nonpolynomial':
        name = 'Nonpoly'

    if exp['dataset']['norm']:
        df_lap_sd[f'D{depth}rmse'][name, noisy]  = exp['lap']['pred_rmse']
        df_lap_sd[f'D{depth}corr'][name, noisy]  = exp['lap']['pred_corr']
        df_gaus_sd[f'D{depth}rmse'][name, noisy] = exp['gaus']['pred_rmse']
        df_gaus_sd[f'D{depth}corr'][name, noisy] = exp['gaus']['pred_corr']

In [ ]:
df_lap_sd

In [ ]:
df_gaus_sd